In [102]:
!pip install pystow==0.4.9 # the most recent version 0.5.0 breaks pykeen, so we use the old version
!pip install git+https://github.com/pykeen/pykeen.git -q

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [103]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

import pykeen

RANDOM_SEED = 42

# Load processed data
os.chdir(r"C:\Users\maxmo\Dropbox\GDS\graph_data_analytics\2025-wt-a2-what-to-watch")

genome_scores = pd.read_csv(r'processed_data\genome_scores_processed_small.csv')
movies_processed = pd.read_csv(r'processed_data\movies_processed.csv')
ratings_train = pd.read_csv(r'processed_data\ratings_train_small.csv')
movies_test_cold_start = pd.read_csv(r'processed_data\movies_test_cold_start.csv')
ratings_test_cold_start = pd.read_csv(r'processed_data\ratings_test_cold_start.csv')
ratings_test = pd.read_csv(r'processed_data\ratings_test_small.csv')
ratings_val_cold_start = pd.read_csv(r'processed_data\ratings_val_cold_start.csv')
ratings_val = pd.read_csv(r'processed_data\ratings_val_small.csv')
# Map user and movie IDs to indices
user_ids = ratings_train['userId'].unique()
movie_ids = movies_processed['movieId'].unique()
user2id = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie2id = {movie_id: idx + len(user_ids) for idx, movie_id in enumerate(movie_ids)}

In [104]:
ratings_test

,userId,movieId,rating,timestamp,positive_rating,negative_rating,year_month,last_rating_year
0,21,750,4.0,2016-02-24 03:43:07,1,0,2016-02,2016-02
1,21,109487,5.0,2016-02-24 03:43:16,1,0,2016-02,2016-02
2,21,293,4.0,2016-02-24 03:43:18,1,0,2016-02,2016-02
3,21,99114,4.5,2016-02-24 03:43:29,1,0,2016-02,2016-02
4,21,68157,4.0,2016-02-24 03:43:34,1,0,2016-02,2016-02
...,...,...,...,...,...,...,...,...
29021,162536,105844,4.5,2019-10-28 10:45:26,1,0,2019-10,2019-11
29022,162536,121231,4.5,2019-10-28 10:45:38,1,0,2019-10,2019-11
29023,162536,1208,5.0,2019-10-29 23:20:36,1,0,2019-10,2019-11
29024,162536,1222,4.0,2019-10-29 23:20:55,1,0,2019-10,2019-11


In [105]:
from pykeen.pipeline import pipeline
from pykeen.models import RESCAL, TransE, DistMult, TransH
from pykeen.evaluation import RankBasedEvaluator
from pykeen.triples import TriplesFactory

def create_triples_factory(ratings, user2id, movie2id):
    """Create a TriplesFactory from a ratings DataFrame."""
    triples = []
    for _, row in ratings.iterrows():
        user_idx = user2id[row['userId']]
        movie_idx = movie2id[row['movieId']]
        if row['rating'] >= 4.0:
            edge_type = 'liked'
        elif row['rating'] < 2.5:
            edge_type = 'disliked'
        else:
            continue  # Skip neutral ratings
        triples.append((user_idx, edge_type, movie_idx))
    triples_np = np.array(triples, dtype=str)
    return TriplesFactory.from_labeled_triples(
        triples=triples_np,
        create_inverse_triples=False
    )

# Create TriplesFactory for training, validation, and testing sets
training_set = create_triples_factory(ratings_train, user2id, movie2id)
validation_set = create_triples_factory(ratings_val, user2id, movie2id)
testing_set = create_triples_factory(ratings_test, user2id, movie2id)

In [106]:
print(type(training_set))
print(type(validation_set))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

<class 'pykeen.triples.triples_factory.TriplesFactory'>
<class 'pykeen.triples.triples_factory.TriplesFactory'>


In [107]:
seen_users = set(ratings_train['userId'].unique())
seen_movies = set(ratings_train['movieId'].unique())
ratings_test_filtered = ratings_test[
    (ratings_test['userId'].isin(seen_users)) &
    (ratings_test['movieId'].isin(seen_movies))
]

ratings_test_filtered['user_idx'] = ratings_test_filtered['userId'].map(user2id)
ratings_test_filtered['movie_idx'] = ratings_test_filtered['movieId'].map(movie2id)

In [108]:
%%time
torch.manual_seed(seed=42)
np.random.seed(seed=42)
transH_train = pipeline(
    model=TransH,
    model_kwargs={"random_seed": 0},
    training=training_set,
    dimensions=64,
    epochs=20,
    random_seed=RANDOM_SEED,
    stopper="early",
    stopper_kwargs={
        "frequency": 5,
        "patience": 10,
        "relative_delta": 0.0,
    },
    validation=validation_set,
    testing=testing_set,
    evaluator=RankBasedEvaluator(),
)

INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\maxmo\.data\pykeen\checkpoints\best-model-weights-80a6aa18-c52a-41b6-8fd1-ec307d0b89a0.pt
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\maxmo\.data\pykeen\checkpoints\best-model-weights-80a6aa18-c52a-41b6-8fd1-ec307d0b89a0.pt
c:\Users\maxmo\anaconda3\envs\GRL_env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as

CPU times: total: 37min 46s
Wall time: 5min 8s
Parser   : 109 ms


In [109]:
%%time
torch.manual_seed(seed=42)
np.random.seed(seed=42)
rescal_train = pipeline(
    model=RESCAL,
    model_kwargs={"random_seed": 0},
    training=training_set,
    dimensions=64,
    epochs=20,
    random_seed=RANDOM_SEED,
    stopper="early",
    stopper_kwargs={
        "frequency": 5,
        "patience": 10,
        "relative_delta": 0.0,
    },
    validation=validation_set,
    testing=testing_set,
    evaluator=RankBasedEvaluator(),
)

INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\maxmo\.data\pykeen\checkpoints\best-model-weights-b2c506ef-2a8a-4e28-803d-6d4af937903d.pt
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\maxmo\.data\pykeen\checkpoints\best-model-weights-b2c506ef-2a8a-4e28-803d-6d4af937903d.pt


Training epochs on cpu:  20%|██        | 4/20 [00:21<01:09,  4.32s/epoch, loss=21.7, prev_loss=22.9]WARNING:pykeen.utils:Using automatic batch size on device.type='cpu' can cause unexplained out-of-memory crashes. Therefore, we use a conservative small batch_size=32. Performance may be improved by explicitly specifying a larger batch size.
INFO:pykeen.evaluation.evaluator:Evaluation took 23.87s seconds
INFO:pykeen.evaluation.evaluator:Evaluation took 23.87s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 5: 0.006817488043957535. Saved model weights to C:\Users\maxmo\.data\pykeen\checkpoints\best-model-weights-b2c506ef-2a8a-4e28-803d-6d4af937903d.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 5.
Training epochs on cpu:  25%|██▌       | 5/20 [00:45<03:11, 12.77s/epoch, loss=21.7, prev_loss=22.9]INFO:pykeen.stoppers.early_stopping:New best result at epoch 5: 0.006817488043957535. Saved model weights to C:\Users\maxmo\.data\pykeen

CPU times: total: 25min 59s
Wall time: 3min 29s


In [110]:
%%time
torch.manual_seed(seed=0)
np.random.seed(seed=0)
transe_train = pipeline(
    model=TransE,
    model_kwargs={"random_seed": 0},
    training=training_set,
    dimensions=64,
    epochs=20,
    random_seed=RANDOM_SEED,
    # early stopping based on the validation set during training
    stopper="early",
    stopper_kwargs={
        "frequency": 5,  # frequency: number of epochs after which an evaluation is made by the stopper
        "patience": 10,  # patience: number of consecutive runs with no improvement, wait until which to stop
        "relative_delta": 0.0,
    },  # relative delta: the minimum increase/decrease in metric to be considered as improvement
    validation=validation_set,
    # evaluate model performance on the test set
    testing=testing_set,
    evaluator=RankBasedEvaluator(),
)

INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.pipeline.api:Using device: None


INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\maxmo\.data\pykeen\checkpoints\best-model-weights-fe47dbae-9e4d-4a69-b130-a0aa1887f96d.pt
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\maxmo\.data\pykeen\checkpoints\best-model-weights-fe47dbae-9e4d-4a69-b130-a0aa1887f96d.pt
Training epochs on cpu:  20%|██        | 4/20 [00:12<00:40,  2.53s/epoch, loss=0.31, prev_loss=0.327] WARNING:pykeen.utils:Using automatic batch size on device.type='cpu' can cause unexplained out-of-memory crashes. Therefore, we use a conservative small batch_size=32. Performance may be improved by explicitly specifying a larger batch size.
INFO:pykeen.evaluation.evaluator:Evaluation took 40.84s seconds
INFO:pykeen.evaluati

CPU times: total: 29min 25s
Wall time: 3min 55s


In [111]:
ratings_test['edge_type'] = ratings_test['rating'].apply(
    lambda x: 'liked' if x >= 4.0 else 'disliked' if x < 2.0 else 'neutral'
)

# Drop rows with 'neutral' edge_type
#ratings_test = ratings_test[ratings_test['edge_type'] != 'neutral']

In [112]:
def predict_edge_type(model, user2id, movie2id, ratings_test):
    predictions = []
    
    # Get ID mappings
    entity_to_id = model.training.entity_to_id
    relation_to_id = model.training.relation_to_id

    for _, row in ratings_test.iterrows():
        user = user2id.get(row['userId'])
        movie = movie2id.get(row['movieId'])
        if user is None or movie is None:
            predictions.append(None)
            continue

        try:
            # Map to numeric IDs
            h_id = entity_to_id[str(user)]
            t_id = entity_to_id[str(movie)]
            liked_id = relation_to_id['liked']
            disliked_id = relation_to_id['disliked']

            # Score the triples
            liked_score = model.model.score_hrt(torch.tensor([[h_id, liked_id, t_id]])).item()
            disliked_score = model.model.score_hrt(torch.tensor([[h_id, disliked_id, t_id]])).item()

            predicted_edge_type = 'liked' if liked_score > disliked_score else 'disliked'
            predictions.append(predicted_edge_type)
        except KeyError:
            predictions.append(None)

    return predictions


In [119]:
def score_edges(model, user_tensor, movie_tensor, ratings_test):
    """
    Scores edges for a batch of user-movie pairs.
    Args:
        model: Trained PyKEEN model.
        user_tensor: Tensor of user IDs.
        movie_tensor: Tensor of movie IDs.
        ratings_test: DataFrame containing test ratings.
    Returns:
        Tensor of scores for the given edges.
    """
    # Get ID mappings
    entity_to_id = model.training.entity_to_id
    relation_to_id = model.training.relation_to_id

    # Map to numeric IDs
    h_ids = user_tensor.cpu().numpy()
    t_ids = movie_tensor.cpu().numpy()
    liked_id = relation_to_id.get('liked')

    # Validate IDs
    valid_h_ids = [h_id for h_id in h_ids if str(h_id) in entity_to_id]
    valid_t_ids = [t_id for t_id in t_ids if str(t_id) in entity_to_id]
    if not valid_h_ids or not valid_t_ids or liked_id is None:
        raise ValueError("Invalid entity or relation IDs detected.")

    # Score the triples
    scores = model.model.score_hrt(
        torch.tensor([[h_id, liked_id, t_id] for h_id, t_id in zip(valid_h_ids, valid_t_ids)], device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    )
    return scores

In [114]:
ratings_test['predicted_edge_type_transe'] = predict_edge_type(transe_train, user2id, movie2id, ratings_test)
ratings_test['predicted_edge_type_transh'] = predict_edge_type(transH_train, user2id, movie2id, ratings_test)
ratings_test['predicted_edge_type_RESCAL'] = predict_edge_type(rescal_train, user2id, movie2id, ratings_test)
# Add predictions for DistMult and RESCAL if trained models are available

In [115]:
# Prepare variables for build_eval_df and evaluate_recommendations
user_movie_test_edges_list = list(zip(
    ratings_test['userId'].map(user2id),
    ratings_test['movieId'].map(movie2id)
))

user_movie_edges_list = list(zip(
    ratings_train['userId'].map(user2id),
    ratings_train['movieId'].map(movie2id)
))

user_movie_val_edges_list = list(zip(
    ratings_val['userId'].map(user2id),
    ratings_val['movieId'].map(movie2id)
))

In [120]:
all_movie_idx = pd.Series(movie_ids).map(movie2id).to_numpy()

def build_eval_df(model,
    test_pos_edges=user_movie_test_edges_list,
    user_movie_edges_list=user_movie_edges_list,
    user_movie_val_edges_list=user_movie_val_edges_list,
    all_movie_ids=all_movie_idx):

    rows = []
    
    # Group the test positives by user
    from collections import defaultdict
    pos_by_user = defaultdict(list)
    for u, m in test_pos_edges:
        pos_by_user[u].append(m)
    
    for user_id, pos_movies in pos_by_user.items():
        # Precompute forbidden set for this user
        forbidden = set(pos_movies) \
                    | {m for (u, m) in user_movie_edges_list    if u == user_id} \
                    | {m for (u, m) in user_movie_val_edges_list if u == user_id}
        
        for pos_m in pos_movies:
            # 1 positive
            batch_movies = [pos_m]
            labels       = [1]
            
            # 9 negatives sampled uniformly
            neg_samples = []
            while len(neg_samples) < 9:
                m = all_movie_ids[torch.randint(len(all_movie_ids), (1,)).item()]
                if m not in forbidden:
                    neg_samples.append(m)
                    forbidden.add(m)  # avoid duplicates
            batch_movies += neg_samples
            labels       += [0] * len(neg_samples)
            
            # Score the batch of 10 edges
            u_tensor  = torch.tensor([user_id] * len(batch_movies), device=device)
            m_tensor  = torch.tensor(batch_movies,      device=device)
            with torch.no_grad():
                scores = score_edges(model, u_tensor, m_tensor, ratings_test_filtered).cpu().numpy()
            
            # Accumulate rows
            for m, s, gt in zip(batch_movies, scores, labels):
                rows.append({
                    'user': user_id,
                    'movie': m,
                    'edge_score': float(s),
                    'ground_truth': gt
                })
    
    # Build DataFrame
    df = pd.DataFrame(rows, columns=['user','movie','edge_score','ground_truth'])
    return df

def evaluate_recommendations(df, ks):
    """
    Expects args:
        df: pandas df with columns ['user', 'movie', 'edge_score', 'ground_truth']
        ks (list of ints): Numbers of top recommendations to consider

    Returns NDCG@k, Recall@k, MRR@k for the given k
    """
    # Group by user
    users = df['user'].unique()
    for k in ks:
        ndcg_list = []
        recall_list = []
        rr_list = []
    
        for user in users:
            user_df = df[df['user'] == user]
            # Sort by predicted score
            ranked = user_df.sort_values('edge_score', ascending=False)
            # Top k predictions
            topk = ranked.head(k)
            # Ground truth relevance values
            rel = topk['ground_truth'].values
            # Compute DCG@k
            gains = (2**rel - 1)
            discounts = np.log2(np.arange(2, k + 2))
            dcg = np.sum(gains / discounts)
    
            # Compute IDCG@k (ideal ranking)
            ideal_rel = np.sort(user_df['ground_truth'].values)[::-1][:k]
            ideal_gains = (2**ideal_rel - 1)
            idcg = np.sum(ideal_gains / discounts)
            ndcg = dcg / idcg if idcg > 0 else 0.0
            ndcg_list.append(ndcg)
    
            # Recall@k: number relevant in topk / total relevant
            total_rel = user_df['ground_truth'].sum()
            recall = rel.sum() / total_rel if total_rel > 0 else 0.0
            recall_list.append(recall)
    
            # MRR@k: reciprocal of rank of the top-ranked true edge
            rr = 0.0
            for idx, val in enumerate(rel, start=1):
                if val == 1:
                    rr = 1.0 / idx
                    break
            rr_list.append(rr)
    
        dict_of_results = {
            f'NDCG@{k}': np.mean(ndcg_list),
            f'Recall@{k}': np.mean(recall_list),
            f'MRR@{k}': np.mean(rr_list)
        }
        # Aggregate metrics
        print(f'METRICS FOR {k}:\n{dict_of_results}')


In [121]:
for model in [transe_train, transH_train, rescal_train]:
    # Build evaluation DataFrame
    df_for_recs = build_eval_df(
        model=model,
        test_pos_edges=user_movie_test_edges_list,
        user_movie_edges_list=user_movie_edges_list,
        user_movie_val_edges_list=user_movie_val_edges_list,
        all_movie_ids=all_movie_idx)
    evaluate_recommendations(df=df_for_recs, ks=[3,5,10])

C:\Users\maxmo\AppData\Local\Temp\ipykernel_2132\2923235229.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'edge_score': float(s),


IndexError: index out of range in self

In [ ]:
for column in ['predicted_edge_type_transe', 'predicted_edge_type_transh', 'predicted_edge_type_RESCAL']:
    agreement = (ratings_test[column] == ratings_test['edge_type']).mean() * 100
    print(f"Percentage agreement for {column}: {agreement:.2f}%")

Percentage agreement for predicted_edge_type_transe: 63.18%
Percentage agreement for predicted_edge_type_transh: 88.72%
Percentage agreement for predicted_edge_type_RESCAL: 46.65%
